In [1]:
pip install pyspark


Note: you may need to restart the kernel to use updated packages.


In [4]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import sum,count
from pyspark.sql.functions import stddev_pop
from pyspark.sql.functions import max
from pyspark.sql.functions import mean ,corr,max, min,stddev_pop ,stddev_samp,covar_pop,covar_samp
spark = SparkSession.builder.getOrCreate()
sc =SparkContext.getOrCreate()

In [11]:
videoStats = spark.read.option("inferschema","true").option("header","true").csv("DEvideos1.csv")
videoStats.show()


+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...|           inscope21|         24|2017-11-13T17:08:...|"inscope21"|"sing...| 252786| 35885|     230|         1539|https://i.ytimg.c...|            False|           Fal

SELECTING SOME OF THE COLUMNS FROM ALL GIVEN COLUMNS

In [15]:
videoStats.createOrReplaceTempView("df")
spark.sql("""select video_id, trending_date, title, views from df""").show(5)

+-----------+-------------+--------------------+-------+
|   video_id|trending_date|               title|  views|
+-----------+-------------+--------------------+-------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...| 252786|
|Bayt7uQith4|     17.14.11|Kinder ferngesteu...| 797196|
|1ZAPwfrtAFY|     17.14.11|The Trump Preside...|2418783|
|AHtypnRk7JE|     17.14.11| Das Fermi-Paradoxon| 380247|
|ZJ9We4bjcg0|     17.14.11|18 SONGS mit Kell...| 822213|
+-----------+-------------+--------------------+-------+
only showing top 5 rows



In [32]:
videoStats=videoStats.select("video_id", "trending_date", "title", "views")
videoStats.show(5)

+-----------+-------------+--------------------+-------+
|   video_id|trending_date|               title|  views|
+-----------+-------------+--------------------+-------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...| 252786|
|Bayt7uQith4|     17.14.11|Kinder ferngesteu...| 797196|
|1ZAPwfrtAFY|     17.14.11|The Trump Preside...|2418783|
|AHtypnRk7JE|     17.14.11| Das Fermi-Paradoxon| 380247|
|ZJ9We4bjcg0|     17.14.11|18 SONGS mit Kell...| 822213|
+-----------+-------------+--------------------+-------+
only showing top 5 rows



ADDING A NEW COLUMN AND FETCHING DATA WITH SOME CONDITION

In [33]:
spark.sql("""select views,views/100 as new from df""").show(5)

+-------+--------+
|  views|     new|
+-------+--------+
| 252786| 2527.86|
| 797196| 7971.96|
|2418783|24187.83|
| 380247| 3802.47|
| 822213| 8222.13|
+-------+--------+
only showing top 5 rows



In [34]:
videoStats.withColumn("new",(col("views")/100)).show(5)

+-----------+-------------+--------------------+------+-------+
|   video_id|trending_date|               title| views|    new|
+-----------+-------------+--------------------+------+-------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...|252786|2527.86|
|Bayt7uQith4|     17.14.11|Kinder ferngesteu...|797196|7971.96|
+-----------+-------------+--------------------+------+-------+
only showing top 2 rows



In [35]:
videoStats.select("*",(col("views")/100).alias("new")).show(5)

+-----------+-------------+--------------------+-------+--------+
|   video_id|trending_date|               title|  views|     new|
+-----------+-------------+--------------------+-------+--------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...| 252786| 2527.86|
|Bayt7uQith4|     17.14.11|Kinder ferngesteu...| 797196| 7971.96|
|1ZAPwfrtAFY|     17.14.11|The Trump Preside...|2418783|24187.83|
|AHtypnRk7JE|     17.14.11| Das Fermi-Paradoxon| 380247| 3802.47|
|ZJ9We4bjcg0|     17.14.11|18 SONGS mit Kell...| 822213| 8222.13|
+-----------+-------------+--------------------+-------+--------+
only showing top 5 rows



FINDING STATISTICS(MEAN,VAR,STD)

In [36]:
spark.sql("""select mean(views) from df """).show()

+---------------------------+
|mean(CAST(views AS DOUBLE))|
+---------------------------+
|          603440.5435958963|
+---------------------------+



In [37]:
videoStats.select(mean("views")).show()

+-----------------+
|       avg(views)|
+-----------------+
|603440.5435958963|
+-----------------+



In [38]:
videoStats.select(max("views")).show()

+----------+
|max(views)|
+----------+
|    999976|
+----------+



In [40]:
videoStats.select(stddev_pop("views")).show()

+-----------------+
|stddev_pop(views)|
+-----------------+
|2348907.150417187|
+-----------------+



GROUPING AND AGGREGATING

In [41]:
videoStats.show()

+-----------+-------------+--------------------+-------+
|   video_id|trending_date|               title|  views|
+-----------+-------------+--------------------+-------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...| 252786|
|Bayt7uQith4|     17.14.11|Kinder ferngesteu...| 797196|
|1ZAPwfrtAFY|     17.14.11|The Trump Preside...|2418783|
|AHtypnRk7JE|     17.14.11| Das Fermi-Paradoxon| 380247|
|ZJ9We4bjcg0|     17.14.11|18 SONGS mit Kell...| 822213|
|xapGFgWqtg4|     17.14.11|Geld verdienen mi...|  32709|
|EIM7RMe39JY|     17.14.11|Kanaken auf der K...| 308683|
|PaWTaj6Iie0|     17.14.11|3 unbekannte Gesi...| 181660|
|GHct2dGNLks|     17.14.11|Antoine leiht Aut...| 369173|
|aZYSFByDGkg|     17.14.11|Legenden: So wird...|  62418|
|2hu_evXPpMM|     17.14.11|Dagi Bee wird Hei...| 228574|
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...| 748374|
|2Zp-Qm3wJkA|     17.14.11|JP Performance - ...| 465883|
|mmMPprcmJeA|     17.14.11|Diese Schlafposit...| 109571|
|3U51cVIqulM|     17.14.11|ARAB

In [42]:
videoStats.groupBy("trending_date").agg(sum("views")).show()

+--------------------+------------+
|       trending_date|  sum(views)|
+--------------------+------------+
| dass sein Geld s...|        null|
|  eine geniale Idee.|        null|
|            18.08.05|1.64752833E8|
|            17.20.11|1.39666442E8|
|            17.09.12|1.73131721E8|
| terrified them a...|        null|
|            18.14.02| 7.7026791E7|
|            18.20.03|1.66007556E8|
|            18.04.05|1.28261998E8|
|            18.21.02|1.44819597E8|
| aber bis heute t...|        null|
|            18.30.04|1.32520295E8|
| dann geht der Sn...|        null|
| plant dieser Xen...|        null|
| dass Hannelore s...|        null|
|    Roti  & Backen: |        null|
|            18.06.05| 1.2446889E8|
| ihrer Großmutter...|        null|
|            18.11.02|1.11605612E8|
| um Viktors Wunde...|        null|
+--------------------+------------+
only showing top 20 rows

